<a href="https://colab.research.google.com/github/santhanreddy864/In-class-exercise-1/blob/main/In_class_exercise_09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The ninth in-class-exercise (20 points in total, 4/16/2021)**

The purpose of the exercise is to practice different machine learning algorithms for text classification as well as the performance evaluation. In addition, you are requried to conduct *10 fold cross validation (https://scikit-learn.org/stable/modules/cross_validation.html)* in the training. 

The dataset can be download from here: https://github.com/unt-iialab/info5731_spring2021/blob/main/class_exercises/exercise09_datacollection.zip. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data. 

Algorithms:

(1) MultinominalNB

(2) SVM 

(3) KNN 

(4) Decision tree

(5) Random Forest

(6) XGBoost

Evaluation measurement:

(1) Accuracy

(2) Recall

(3) Precison 

(4) F-1 score

**Read and Train Dataset**

In [3]:
import pandas as pd
data = open('/content/stsa-train.txt').read()
labels, t = [], []
for i, line in enumerate(data.split("\n")):
    content = line.split(' ')
    labels.append(content[0])
    t.append(" ".join(content[1:]))
t_data = open('/content/stsa-test.txt').read()
t_labels, t_text = [], []
for i, line in enumerate(t_data.split("\n")):
    content = line.split(' ')
    t_labels.append(content[0])
    t_text.append(" ".join(content[1:]))


**Create Dataframe**

In [12]:
df = pd.DataFrame (list(zip(text, labels)) , columns = ['Raw Text', 'Labels'])
ts_df = pd.DataFrame (list(zip(t_text, t_labels)) , columns = ['Raw Text', 'Labels'])
ts_df = test_df.dropna()

**Downloading Packages**

In [9]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package brown to /root/nltk_data...
       |   Unzipping corpora/brown.zip.
       | Downloading package brown_tei to /root/nltk_data...
       |   Unzipping corpora/brown_tei.zip.
       | Downloading package cess_cat to /root/nltk_data...
       |   Unzipping corpora/cess_cat.zip.
       | Downloading package cess_esp to /root/nltk_data...
 

True

In [13]:
import nltk
import re
from nltk.stem import PorterStemmer
from textblob import Word
from nltk.corpus import stopwords
from textblob import TextBlob
nltk.download('wordnet')
nltk.download('punkt')
stop = stopwords.words('english')
st = PorterStemmer()
df['Lower Case'] = df['Raw Text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['Punctuation Removal'] = df['Lower Case'].str.replace('[^\w\s]','')
df['Special Characters Removal'] = df['Punctuation Removal'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))
df['Remove Stopwords'] = df['Punctuation Removal'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['Tokenization'] = df['Remove Stopwords'].apply(lambda x: TextBlob(x).words)
df['Lemmatization'] = df['Tokenization'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))
ts_df['Lower Case'] = test_df['Raw Text'].apply(lambda x: " ".join(x.lower() for x in x.split()))
ts_df['Punctuation Removal'] = test_df['Lower Case'].str.replace('[^\w\s]','')
ts_df['Special Characters Removal'] = test_df['Punctuation Removal'].apply(lambda x: ''.join(re.sub(r"[^a-zA-Z0-9]+", ' ', charctr) for charctr in x ))
ts_df['Remove Stopwords'] = test_df['Punctuation Removal'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
ts_df['Tokenization'] = test_df['Remove Stopwords'].apply(lambda x: TextBlob(x).words)
ts_df['Lemmatization'] = test_df['Tokenization'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x]))


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


**Transforming Data**

In [18]:
from sklearn import model_selection, preprocessing, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(analyzer='word')
tfidf_vect.fit(df['Lemmatization'])
x_tfidf =  tfidf_vect.transform(df['Lemmatization'])
vect_ts = TfidfVectorizer(analyzer='word', vocabulary = tfidf_vect.vocabulary_)
vect_ts.fit(test_df['Lemmatization'])
x_ts = vect_ts.transform(test_df['Lemmatization'])
y_ts = test_df['Labels']

**Data Splitting** 

In [58]:
from sklearn import model_selection, preprocessing, naive_bayes, metrics, svm
tr_x, va_x, tr_y, va_y = model_selection.train_test_split(x_tfidf, df['Labels'].values,test_size=0.2)
from sklearn import metrics
def get_metrics(predictions, test_data_y):
  accuracy = metrics.accuracy_score(predictions, test_data_y)
  precision = metrics.precision_score(predictions, test_data_y, pos_label='positive', average='micro')
  recall = metrics.recall_score(predictions, test_data_y, pos_label='positive', average='micro')
  f1 = metrics.f1_score(predictions, test_data_y, pos_label='positive', average='micro')
  return accuracy, precision, recall, f1
import numpy as np
from sklearn.model_selection import cross_val_score, KFold
def cross_score(modelName, x, y):
  scoring = 'accuracy'
  kfold = KFold(10, random_state = 7,shuffle=True)
  cross_val = cross_val_score(modelName, x, y, cv=kfold).mean()
  return cross_val


**Evaluate models**

**MultinominalNB**

In [61]:
nb_model = naive_bayes.MultinomialNB()
nb_model.fit(train_x, train_y)
nb_predictions_validation_data = nb_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(nb_predictions_validation_data, valid_y)
print("Accuracy ={0}\nPrecision = {1}\nRecall = {2}\nF1 = {3}".format(accuracy, precision, recall, f1))

nb_scores_mean = cross_score(nb_model, valid_x, valid_y)
print('\nCross Score for Navie Bayes = {0}'.format(nb_scores_mean))

Accuracy =0.7689530685920578
Precision = 0.7689530685920578
Recall = 0.7689530685920578
F1 = 0.7689530685920578

Cross Score for Navie Bayes = 0.7126316338233761


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


**Test**

In [62]:
nb_predictions_test_data = nb_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(nb_predictions_test_data, test_y)
print("Accuracy = {0}\nPrecision = {1}\nRecall = {2}\nF1 = {3}".format(accuracy, precision, recall, f1))

nb_test_scores_mean = cross_score(nb_model, xtest, test_y)
print('\nCross Score for Navie bayes = {0}'.format(nb_test_scores_mean))

Accuracy = 0.7859495060373216
Precision = 0.7859495060373216
Recall = 0.7859495060373216
F1 = 0.7859495060373216

Cross Score for Navie bayes = 0.7371044256290158


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


**SVM**

In [40]:
svm_model = svm.SVC()
svm_model.fit(train_x, train_y)
svm_predictions_validation_data = svm_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(svm_predictions_validation_data, valid_y)
print("Accuracy = {0}\nPrecision = {1}\nRecall = {2}\nF1 = {3}".format(accuracy, precision, recall, f1))

svm_scores_mean = cross_score(svm_model, valid_x, valid_y)
print('\nCross Score for SVM = {0}'.format(svm_scores_mean))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy = 0.7581227436823105
Precision = 0.7581227436823105
Recall = 0.7581227436823105
F1 = 0.7581227436823105

Cross Score for SVM = 0.6967573767073298


**Test**

In [42]:
svm_predictions_test_data = svm_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(svm_predictions_test_data, test_y)
print("Accuracy = {0}\nPrecision = {1}\nRecall = {2}\nF1 = {3}".format(accuracy, precision, recall, f1))

svm_test_scores_mean = cross_score(svm_model, xtest, test_y)
print('\nCross Score for SVM {0}'.format(svm_test_scores_mean))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy = 0.7892425905598244
Precision = 0.7892425905598244
Recall = 0.7892425905598244
F1 = 0.7892425905598244

Cross Score for SVM 0.7239086050561461


**KNN**

In [44]:
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier(n_neighbors = 15)
knn_model.fit(train_x, train_y)
knn_predictions_valid_data = knn_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(knn_predictions_valid_data, valid_y)
print("Accuracy = {0}\nPrecision = {1}\nRecall = {2}\nF1 = {3}".format(accuracy, precision, recall, f1))

knn_scores_mean = cross_score(knn_model, valid_x, valid_y)
print('\nCross Score for KNN = {0}'.format(knn_scores_mean))


Accuracy = 0.7083032490974729
Precision = 0.7083032490974729
Recall = 0.7083032490974729
F1 = 0.708303249097473

Cross Score for KNN = 0.6902304243561673


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


**TEST**

In [45]:
knn_predictions_test_data = knn_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(knn_predictions_test_data, test_y)
print("Accuracy is {0}\nPrecision is {1}\nRecall is {2}\nF1 is {3}".format(accuracy, precision, recall, f1))

knn_test_scores_mean = cross_score(knn_model, xtest, test_y)
print('\nCrossScore for KNN = {0}'.format(knn_test_scores_mean))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy is 0.7156970362239298
Precision is 0.7156970362239298
Recall is 0.7156970362239298
F1 is 0.7156970362239298

CrossScore for KNN = 0.6361376328589443


**Decision Tree**

In [25]:
from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier()
dt_model.fit(train_x, train_y)
dt_predictions_valid_data = dt_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(dt_predictions_valid_data, valid_y)
print("Accuracy = {0}\nPrecision = {1}\nRecall = {2}\nF1 = {3}".format(accuracy, precision, recall, f1))

dt_scores_mean = cross_score(dt_model, valid_x, valid_y)
print('Cross  Score for desicion tree = {0}'.format(dt_scores_mean))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy = 0.6577617328519856
Precision = 0.6577617328519856
Recall = 0.6577617328519856
F1 = 0.6577617328519856
Cross  Score for desicion tree = 0.5775622979876968


**Test**

In [26]:
dt_predictions_test_data = dt_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(dt_predictions_test_data, test_y)
print("Accuracy = {0}\nPrecision =  {1}\nRecall = {2}\nF1 = {3}".format(accuracy, precision, recall, f1))

dt_test_scores_mean = cross_score(dt_model, xtest, test_y)
print('Cross  Score for desicion tree = {0}'.format(dt_test_scores_mean))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy = 0.6608122941822173
Precision =  0.6608122941822173
Recall = 0.6608122941822173
F1 = 0.6608122941822173
Cross  Score for desicion tree = 0.644331351708401


**Random Forest**

In [47]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier()
rf_model.fit(train_x, train_y)
rf_predictions_valid_data = rf_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(rf_predictions_valid_data, valid_y)
print("Accuracy = {0}\nPrecision = {1}\nRecall = {2}\nF1 = {3}".format(accuracy, precision, recall, f1))

rf_scores_mean = cross_score(rf_model, valid_x, valid_y)
print('\nCross Score for Random Forest{0}'.format(rf_scores_mean))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy = 0.711913357400722
Precision = 0.711913357400722
Recall = 0.711913357400722
F1 = 0.711913357400722

Cross Score for Random Forest0.638947972057137


**TEST**

In [50]:
rf_predictions_test_data = rf_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(rf_predictions_test_data, test_y)
print("Accuracy = {0}\nPrecision = {1}\nRecall = {2}\nF1 = {3}".format(accuracy, precision, recall, f1))

rf_test_scores_mean = cross_score(rf_model, xtest, test_y)
print('\nCross Score for Random Forest =  {0}'.format(rf_test_scores_mean))

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy = 0.7409440175631175
Precision = 0.7409440175631175
Recall = 0.7409440175631175
F1 = 0.7409440175631175

Cross Score for Random Forest =  0.6756380231790067


**XG Boost**

In [52]:
from xgboost import XGBClassifier
xg_model = XGBClassifier()
xg_model.fit(train_x, train_y)
xg_predictions_valid_data = xg_model.predict(valid_x)
accuracy, precision, recall, f1 = get_metrics(xg_predictions_valid_data, valid_y)
print("Accuracy = {0}\nPrecision = {1}\nRecall = {2}\nF1 = {3}".format(accuracy, precision, recall, f1))

xg_scores_mean = cross_score(XGBClassifier(), valid_x, valid_y)
print('\nCross Score for XG Boost {0}'.format(xg_scores_mean))


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy = 0.6057761732851985
Precision = 0.6057761732851985
Recall = 0.6057761732851985
F1 = 0.6057761732851985

Cross Score for XG Boost 0.594885830466062


**TEST**

In [54]:
xg_predictions_test_data = xg_model.predict(xtest)
accuracy, precision, recall, f1 = get_metrics(xg_predictions_test_data, test_y)
print("Accuracy = {0}\nPrecision = {1}\nRecall = {2}\nF1 = {3}".format(accuracy, precision, recall, f1))

xg_test_scores_mean = cross_score(XGBClassifier(), xtest, test_y)
print('\nCross Score for XG Boost = {0}'.format(xg_test_scores_mean))


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1321: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'micro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)


Accuracy = 0.6322722283205269
Precision = 0.6322722283205269
Recall = 0.6322722283205269
F1 = 0.6322722283205269

Cross Score for XG Boost = 0.6251035849396506


**Evaluation measurement for model accuracies on test data:**

Navie Bayes = 78.59% , 
SVM = 78.92% , 
KNN = 71.56% , 
Decision Tree = 66.08% , 
Random Forest = 74.09% , 
XG Boost = 63.27% , 